# Spelling corrector challenge

In this exercise, we deal with  unigram language model. Our goal is to get a spelling corrector with a good accuracy and process speed.

In [3]:
import re
from collections import Counter

In [4]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

While fast typing, it is more likely to make a mistake in adding a character than replacing. The idea is to give a higher probability to longer words. The accuracy improved the most (about 1-5%) using an exponential scale, which confirm our hypothesis. People do make many more errors when typing longer words.

In [30]:
# Language model : estimate the probability of a word
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word]*10**(len(word))/ N

# Error model 
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word),key=P)

In [10]:
N=sum(WORDS.values())
N

1115585

In [87]:
# Candidate model function
def candidates(word):
    "Generate possible spelling corrections for word."
    return (known(google_checker(word))or known(insert_rst(word)) or known(duplicate_edit (word)) or  known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [33]:
# Restrict words to known
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [34]:
# Candidate model of the baseline

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Improvement on Candidate model

Specify typing errors in our candidate model and see if it improves our accuracy.

Most of the time, a misspelled word looks similar to the actual word. It is very unlikely that someone missing 4 characters in a word. It is also to long to consider the more than 3 edit errors cases. Therefore, we make allowance for at most 2 mistakes - 2 edit distance away. This helps us maximising our accuracy. In order to improve our model, we need to think about which are the most common mistakes : For example, it is a common mistake to repeat a same character, to insert a letter or to omit repetition of letters. 

In [35]:
# Add the edits number away from word : I abandon this idea, it is not that common to do 3 edit errors in a word 
#def edits3(word):
#    "All edits that are three edits away from `word."
#    return (e3 for e2 in edits2(word) for e3 in edits2(e2))

In [36]:
#It is more common to do duplicate error, by adding duplicate function we improved the speed of our corrector.
def duplicate_edit (word):
    "Duplicates letters"
    splits = [(word[:i], word[i:])for i in range(len(word) + 1)]
    inserts = [L + R[0] + R for L, R in splits if R]
    return set(inserts)

# we can try to only consider most common letters in English
def insert_rst(word):
    letters    = 'rst'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(inserts)

The following shows some attemps that I think will improve the model, but turn out to give little imporvement or even make the model less accurate, so I did not include them in the candidate model.

In [ ]:
#delete or insert a letter
def delete_edit(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def insert_edit(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(inserts)

# swap letters  
def trans2_edit(word):
    letters     = 'abcdefghijklmnopqrstuvwxyz'
    splits      = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    transposes0 = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    transposes1 = [L + R[0] + R[2:] for L, R in splits if len(R)>1]
    transposes2 = [L + R[1] + R[2:] for L, R in splits if len(R)>1]
    return set(transposes0 + transposes1 + transposes2 )

# vowel mistyping is a common mistake 
def vowels_edit(word):
    vowels = ['a','e','i','o','u','y']
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]          for L, R in splits if R for c in vowels]
    return set(replaces )


The following shows some other attemps that can be used in the model. But adding with other functions (especially with duplicate_edit), they do not present much improvement , so I did not include in the candidate model.

In [37]:
def similar(char):
    "Generates some similar characters if char satisfies any of conditions"
    if (char == 'c'):
        return 's'
    if (char == 's'):
        return 'c'
    if (char == 'b'):
        return 'p'
    if (char == 'p'):
        return 'b'
    if (char == 'n'):
        return 'm'
    if (char == 'm'):
        return 'n'
    if (char == 'd'):
        return 't'
    if (char == 't'):
        return 'd'
    else:
        return ''
    
def similar_edit(word):
    "Getting set of where chars are replaced by similar ones (if any) (common mictake?)"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]   for L, R in splits if R for c in similar(R[0])]
    return set(replaces)

def close(char):
    "Generates the most common consonants R S T for some close on the keyboard characters if char satisfies any of conditions"
    if (char == 'a'):
        return 's'
    if (char == 'w'):
        return 's'
    if (char == 'd'):
        return 's'
    if (char == 'z'):
        return 's'
    if (char == 'x'):
        return 's'
    if (char == 'e'):
        return 'r'
    if (char == 'f'):
        return 'r'
    if (char == 'g'):
        return 't'
    if (char == 'y'):
        return 't'
    else:
        return ''

def close_edit(word):
    "Getting set of where chars are replaced by closed ones (if any)"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]     for L, R in splits if R for c in close(R[0])]
    return set(replaces)

def rare(char):
    "Generates some close on the keyboard characters for the rarest letters in english J Q X Z"
    if (char == 'j'):
        return 'k'
    if (char == 'j'):
        return 'h'
    if (char == 'j'):
        return 'u'
    if (char == 'j'):
        return 'm'
    if (char == 'j'):
        return 'n'
    if (char == 'j'):
        return 'm'
    if (char == 'q'):
        return 'w'
    if (char == 'q'):
        return 'a'
    if (char == 'x'):
        return 'c'
    if (char == 'x'):
        return 'd'
    if (char == 'z'):
        return 'a'
    else:
        return ''
    
def rare_edit(word):
    "Getting set of where chars are replaced by closed ones (if any)"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    replaces   = [L + c + R[1:]     for L, R in splits if R for c in rare(R[0])]
    
    return set(replaces)


# Google checker

This is a program that I found on the internet that turned out to be very useful. It obtains the corrected word checked from Google and check if such word exists in big.txt.

In [89]:
import  requests, re
from html.parser import HTMLParser
import urllib3
class GoogleSpellCheckCommand(object):
    
    def __init__(self):
        "Construtor"
        self.manager = urllib3.PoolManager()

    def correct(self, text):
        "Correct input text by using Google Spell Corrector"
        
        # grab html
        html = self.get_page('http://www.google.com/search?hl=en&q=' + requests.utils.quote(text) + "&meta=&gws_rd=ssl")
        html_parser = HTMLParser()

        # save html for debugging
        # open('page.html', 'w').write(html)

        # pull pieces out
        match = re.search(r'(?:Showing results for|Did you mean|Including results for)[^\0]*?<a.*?>(.*?)</a>', html)
        if match is None:
            fix = text
        else:
            fix = match.group(1)
            fix = re.sub(r'<.*?>', '', fix)
            fix = html_parser.unescape(fix)
        # return result
        return fix

    def get_page(self, url):
        # the type of header affects the type of response google returns
        # for example, using the commented out header below google does not 
        # include "Including results for" results and gives back a different set of results
        # than using the updated user_agent yanked from chrome's headers
        # user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'
        headers = {'User-Agent':user_agent,}

        req =  self.manager.request('GET', url, headers)
        
        return str(req.data)

In [90]:
def google_checker(word):
    google_corrector = GoogleSpellCheckCommand()
    
    return [google_corrector.correct(word)]

I also wanted to try pyEnchant. It is a python package for spell checking and correcting that help to get similar words suggestion. But I had trouble in getting the enchant library installed. But I think it will successfully improve the accuracy of the model, by checking if the most likely words are in big.txt.

# Test code

In [40]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

In [41]:
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [91]:
if __name__ == '__main__':
    spelltest(Testset(open('spell-testset1.txt')))
    spelltest(Testset(open('spell-testset2.txt')))
    spelltest(Testset(open('aspell.txt')))
    spelltest(Testset(open('wikipedia.txt')))
   

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


86% of 270 correct (6% unknown) at 45 words per second 
79% of 400 correct (11% unknown) at 35 words per second 
56% of 531 correct (23% unknown) at 19 words per second 
67% of 2455 correct (24% unknown) at 22 words per second 


Here, I get around 10% better in accuracy, with a better precessing words speed : 5 to 12 words/second better. In order to do better, we can try to add some costs to edits in the error model to penalize rare mistakes, and continue to improve our correction model : for example by adding a limited set of words at edit distance 3. If we want to go further, we will want to consider surrounding words, which will help a lot in selecting the right word.